In [1]:
from rdflib import URIRef, BNode, Literal, Graph, plugin, Namespace
from rdflib.serializer import Serializer 
from rdflib.namespace import FOAF, DC, RDF, RDFS, OWL, SKOS, NamespaceManager
import csv

In [5]:
kgcBase = Graph()
kgcBase.parse('./ontology.ttl', format='ttl')

<Graph identifier=N192c26b11f3545ef85cea54ea1351c9d (<class 'rdflib.graph.Graph'>)>

In [6]:
# Cases to confirm

nmGraph = NamespaceManager(kgc2020)
schema = Namespace("http://schema.org/")
nmGraph.bind("schema", schema)
wd = Namespace("http://www.wikidata.org/entity/")
nmGraph.bind("wd", wd)
owl = Namespace("http://www.w3.org/2002/07/owl#")
nmGraph.bind("owl", owl)
skos = Namespace("http://www.w3.org/2004/02/skos/core#")
nmGraph.bind("skos", skos)
kgc = Namespace("http://www.knowledgegraph.tech/iri/")
nmGraph.bind("kgc", kgc)
owl = Namespace("http://www.w3.org/2002/07/owl#")
nmGraph.bind("owl", owl)
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
nmGraph.bind("rdf", rdf)
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
nmGraph.bind("rdfs", rdfs)
xml = Namespace("http://www.w3.org/XML/1998/namespace")
nmGraph.bind("xml", xml) 
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
nmGraph.bind("xsd", xsd) 

## Initializations

In [7]:
# Initializations 
startFrom = 0
lastId = startFrom + 1
labelDict = {}

kgcGraph2020=kgcBase

## Conference Information

In [ ]:
# What about event series vs conference/workshop - Fix ontology if must - Perhaps both are sub types of series
# Also problem with IRI or not

KGC2020 = URIRef(("http://www.knowledgegraph.tech/conference-2020"))

kgcGraph2020.add((wd.Q76451254, RDF.type, kgc.Wiki))
kgcGraph2020.add((KGC2020, owl.sameAs, wd.Q76451254))
kgcGraph2020.add((KGC2020, schema.sameAs, wd.Q76451254))

kgcGraph2020.add((KGC2020, schema.name, Literal("Knowledge Graph Conference 2020")))
kgcGraph2020.add((KGC2020, RDF.type, schema.EventSeries))


kgcGraph2020.add((KGC2020, schema.eventAttendanceMode, schema.OnlineEventAttendanceMode))

## Speakers

In [ ]:
with open('KGC-2020-Speakers-Recon.csv', mode='r') as csv_file:
    speaker_file = csv.DictReader(csv_file)


    for row in speaker_file:
        
        
        # Speaker's Name ---------------------------------------------------------------------------------------
        name = row['Name']
        if name:
            if name in labelDict:
                speaker = labelDict[name]
                #print("Existing Name:", name)
            else:
                newId = str(lastId).zfill(5)
                speaker = URIRef(kgc + newId)
                #print("Else Section!", name, speaker)
                lastId += 1
                #print("Incremented Last ID is ", lastId)
                kgcGraph2020.add((speaker, RDF.type, schema.Person))
                kgcGraph2020.add((speaker, schema.name, Literal(name)))
                
                if row["personQID"]:
                    #print("personQID not empty!")
                    for qid in row["personQID"].split(';'):
                        #print("Qid is ", qid)
                        kgcGraph2020.add((speaker, OWL.sameAs, URIRef(wd + qid.strip())))
                labelDict[name] = speaker
         
        
        if row["LinkedIn"]:
            #print("LinkedIn not empty!")
            kgcGraph2020.add((speaker, owl.sameAs, URIRef(row["LinkedIn"])))
         
               
        # Speaker's Country of Citizenship -------------------------------------------------------------------------- 
        countryName = row["countryOfCitizenship"]
        if countryName:
            if countryName in labelDict:
                country = labelDict[countryName]
            else:
                newId = str(lastId).zfill(5)
                country = URIRef(kgc + newId)
                #print("Else Section!", country, countryName)
                lastId += 1
                kgcGraph2020.add((country, RDF.type, schema.Country))
                kgcGraph2020.add((country, schema.name, Literal(countryName)))
            
                if row["countryOfCitizenshipQID"]:
                    #print("country of citizenship QID empty!")
                    kgcGraph2020.add((country, OWL.sameAs, URIRef(wd + row["countryOfCitizenshipQID"])))
                labelDict[countryName] = country
            kgcGraph2020.add((speaker, schema.nationality, country)) 
            
            
        # Speaker's School Name  -------------------------------------------------------------------------- 
        schoolName = row["educatedAt"]
        if schoolName:
            if schoolName in labelDict:
                school = labelDict[schoolName]
            else:
                newId = str(lastId).zfill(5)
                school = URIRef(kgc + newId)
                #print("Else Section!", school, schoolName)
                lastId += 1
                kgcGraph2020.add((school, schema.name, Literal(schoolName)))
                kgcGraph2020.add((school, RDF.type, schema.CollegeOrUniversity))
                if row["educatedAtQID"]:
                    #print("education At QID not empty!")
                    kgcGraph2020.add((school, OWL.sameAs, URIRef(wd + row["educatedAtQID"])))
                labelDict[schoolName] = school
            kgcGraph2020.add((speaker, schema.alumniOf, school))
    
    
    
        # Person's field of Work  -------------------------------------------------------------------------- 
        pfowName = row["personFieldOfWork"]
        if pfowName:
            #print("personFieldOfWork not empty!")
            if pfowName in labelDict:
                pfow = labelDict[pfowName]
            else:
                newId = str(lastId).zfill(5)
                pfow = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((pfow, schema.name, Literal(pfowName)))
                kgcGraph2020.add((pfow, RDF.type, skos.Concept))
                if row["personFieldOfWorkQID"]:
                    #print("personField QID not empty!")
                    kgcGraph2020.add((pfow, OWL.sameAs, URIRef(wd + row["personFieldOfWorkQID"])))
                labelDict[pfowName] = pfow
            kgcGraph2020.add((speaker, schema.knowsAbout, pfow))

            
       
        # Person's Company   -------------------------------------------------------------------------- 
        # Question. Local Business or ????????????
        coName = row["Company"]
        if coName:
            #print("company not empty!")
            if coName in labelDict:
                co = labelDict[coName]
            else:
                newId = str(lastId).zfill(5)
                co = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((co, schema.name, Literal(coName)))
                kgcGraph2020.add((co, RDF.type, schema.Organization))
                if row["companyQID"]:
                    #print("Company QID not empty!")
                    for qid in row["companyQID"].split(';'):
                        kgcGraph2020.add((co, OWL.sameAs, URIRef(wd + qid.strip())))
                labelDict[coName] = co
            kgcGraph2020.add((speaker, schema.worksFor, co))
            
            
            
        # Consortime Name   --------------------------------------------------------------------------  
        consortiumName = row["member of"]
        if consortiumName:
            #print("member of not empty!")
            if consortiumName in labelDict:
                consortium = labelDict[consortiumName]
            else:
                newId = str(lastId).zfill(5)
                consortium = URIRef(kgc + newId)
                lastId += 1                
                kgcGraph2020.add((consortium, schema.name, Literal(consortiumName)))
                kgcGraph2020.add((consortium, RDF.type, schema.Organization))
                if row["memberOfQID"]:
                    #print("member of QID not empty!")
                    kgcGraph2020.add((consortium, OWL.sameAs, URIRef(wd + row["memberOfQID"])))
                labelDict[consortiumName] = consortium
            kgcGraph2020.add((co, schema.memberOf, consortium))
            
        

        # HQ Name   -------------------------------------------------------------------------------   
        # Quetions ----------- What are the below fields? Address, city, what?
        # Not ready for running!!!!!!!!
        
        AddressName = row["HQ"]
        if AddressName:
            #print("HQ not empty!")
            if AddressName in labelDict:
                Address = labelDict[AddressName]
            else:
                newId = str(lastId).zfill(5)
                Address = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((Address, schema.name, Literal(HQName)))
                kgcGraph2020.add((HQ, RDF.type, schema.Place))
                if row["HQQID1"]:
                    #print("HQID 1 not empty!")
                    kgcGraph2020.add((HQ, OWL.sameAs, URIRef(wd + row["HQQID1"])))
                labelDict[HQName] = HQ
            kgcGraph2020.add((co, schema.location, HQ))  
            
            
        # HQ2 Name   -------------------------------------------------------------------------------   
        CityName = row["HQ2"]
        if CityName:
            #print("HQ2 not empty!")
            if CityName in labelDict:
                City = labelDict[CityName]
            else:
                newId = str(lastId).zfill(5)
                City = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((HQ2, schema.name, Literal(CityName)))
                kgcGraph2020.add((HQ2, RDF.type, schema.City))
                if row["HQQID2"]:
                    #print("HQ ID2 not empty!")
                    kgcGraph2020.add((HQ2, OWL.sameAs, URIRef(wd + row["HQQID2"])))
                labelDict[HQ2Name] = HQ2
            kgcGraph2020.add((HQ, schema.geoWithin, HQ2))
            
            
        # HQ3 Name   -------------------------------------------------------------------------------   
        HQ3Name = row["HQ3"]
        if HQ3Name:
            #print("HQ3 not empty!")
            if HQ3Name in labelDict:
                HQ3 = labelDict[HQ3Name]
            else:
                newId = str(lastId).zfill(5)
                HQ3 = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((HQ3, schema.name, Literal(HQ3Name)))
                kgcGraph2020.add((HQ3, RDF.type, schema.Place))
                if row["HQQID3"]:
                    #print("HQID 3 not empty!")
                    kgcGraph2020.add((HQ3, OWL.sameAs, URIRef(wd + row["HQQID3"])))
                labelDict[HQ3Name] = HQ3
            kgcGraph2020.add((HQ2, schema.geoWithin, HQ3))
            
        
        # HQ4 Name   -------------------------------------------------------------------------------   
        HQ4Name = row["HQ4"]
        if HQ4Name:
            #print("HQ4 not empty!")
            if HQ4Name in labelDict:
                HQ4 = labelDict[HQ4Name]
            else:
                newId = str(lastId).zfill(5)
                HQ4 = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((HQ4, schema.name, Literal(HQ4Name)))
                kgcGraph2020.add((HQ4, RDF.type, schema.Place))
                if row["HQQID4"]:
                    #print("HQ ID4 not empty!")
                    kgcGraph2020.add((HQ4, OWL.sameAs, URIRef(wd + row["HQQID4"])))
                labelDict[HQ4Name] = HQ4
            kgcGraph2020.add((HQ3, schema.geoWithin, HQ4))
            
        
        # HQ5 Name   ------------------------------------------------------------------------------- 
        HQ5Name = row["HQ5"]
        #print("The type of Hq5 is ",type(HQ5Name))
        #if not ((HQ5Name is None) or (HQ5Name == "")):
        if HQ5Name:
            #print("HQ5 is not empty")
            if HQ5Name in labelDict:
                HQ5 = labelDict[HQ5Name]
            else:
                newId = str(lastId).zfill(5)
                HQ5 = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((HQ5, schema.name, Literal(HQ5Name)))
                kgcGraph2020.add((HQ5, RDF.type, schema.Place))
                if row["HQQID5"]:
                    #print("HQQID5 is not empty!")
                    kgcGraph2020.add((HQ5, OWL.sameAs, URIRef(wd + row["HQQID5"]))) # Watch this!
                labelDict[HQ5Name] = HQ5
            kgcGraph2020.add((HQ4, schema.geoWithin, HQ5))
            
        
        # Additional Type Name   -------------------------------------------------------------------------------
        additionalTypeName = row["instance of"]
        if additionalTypeName:
             #print("instance of not empty!")
            if additionalTypeName in labelDict:
                additionalType = labelDict[additionalTypeName]
            else:
                newId = str(lastId).zfill(5)
                additionalType = URIRef(kgc + newId)
                lastId += 1 
                kgcGraph2020.add((additionalType, schema.name, Literal(additionalTypeName)))
                kgcGraph2020.add((additionalType, RDF.type, owl.class))
                if row["instanceOfQID"]: 
                    #print("instance of QID not empty!")
                    kgcGraph2020.add((additionalType, OWL.sameAs, URIRef(wd + row["instanceOfQID"])))
                labelDict[additionalTypeName] = additionalType
            kgcGraph2020.add((co, schema.additionalType, additionalType))
            
          
         # Industry Name   -------------------------------------------------------------------------------
        industryName = row["industry"]
        if industryName:
            #print("industry Name not empty!")
            if industryName in labelDict:
                industry = labelDict[industryName]
            else:
                newId = str(lastId).zfill(5)
                industry = URIRef(kgc + newId)
                lastId += 1
                kgcGraph2020.add((industry, schema.name, Literal(industryName)))
                kgcGraph2020.add((industry, RDF.type, schema.Thing))
                if row["industryQID"]:
                    #print("industry QID not empty!")
                    kgcGraph2020.add((industry, OWL.sameAs, URIRef(wd + row["industryQID"])))
                labelDict[industryName] = industry
            kgcGraph2020.add((co, schema.knowsAbout, industry))          
            
        

    